In [8]:
import pandas as pd
import requests
import bs4
import time
import re
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [12]:
euro = pd.read_csv('eurovision_winners_clean.csv')
def clean_year(val):
    val = str(val)
    if val[:4].isdigit():
        return val[:4]
    return None 
euro['Cleaned_Year'] = euro['Year'].apply(clean_year)
euro = euro.drop('Year', axis=1)
euro

,Country,Song,Artist,Points,Cleaned_Year
0,—,—,NaN,—,2025
1,Switzerland,The Code,Nemo,591,2024
2,Sweden,Tattoo,Loreen,583,2023
3,Ukraine,Stefania,Kalush Orchestra,631,2022
4,Italy,Zitti e buoni,Måneskin,524,2021
...,...,...,...,...,...
65,France,Tom Pillibi,Jacqueline Boyer,32,1960
66,Netherlands,Een beetje,Teddy Scholten,21,1959
67,France,Dors mon amour,André Claveau,27,1958
68,Netherlands,Net als toen,Corry Brokken,31,1957


In [ ]:
for i in range(len(euro)):
    year = euro.iloc[1]['Cleaned_Year']
    country = euro.iloc[1]['Country']


Year         2024Malmö
Country    Switzerland
Song          The Code
Artist            Nemo
Points             591
Name: 1, dtype: object

In [59]:
driver = webdriver.Chrome()

def scrape_lyrics(year, country):
    try:
        url = f'https://eurovisionworld.com/eurovision/{year}/{country}'
        driver.get(url)

        src = driver.page_source
        parser = bs4.BeautifulSoup(src, 'html.parser')

        data = parser.find('div', attrs={'class': 'lyrics_div', 'data-lyrics-version': 'English'})
        para = data.find_all('p')
        lyrics = '\n'.join([re.sub(r'<p[^>]*>', '', str(i).replace('<br/>', '\n').replace('</p>', '')) for i in para])
        return lyrics
    except:
        return 'none'

    # request = requests.get(url)
    # data = bs4.BeautifulSoup(request.text, 'html.parser')
    # l = data.find_all('div', attrs={'class':'top'}) #, 'data-lyrics-version': 'English'})
    # print(l)


lyrics_list = []
for i in range(len(euro)):
    year = euro.iloc[i]['Cleaned_Year']
    country = euro.iloc[i]['Country']
    print(year + country)
    lyrics_list.append(scrape_lyrics(year, country))

print(lyrics_list)
driver.close()
    


2025—
2024Switzerland
2023Sweden
2022Ukraine
2021Italy
2020—
2019Netherlands
2018Israel
2017Portugal
2016Ukraine
2015Sweden
2014Austria
2013Denmark
2012Sweden
2011Azerbaijan
2010Germany
2009Norway
2008Russia
2007Serbia
2006Finland
2005Greece
2004Ukraine
2003Turkey
2002Latvia
2001Estonia
2000Denmark
1999Sweden
1998Israel
1997United KingdomUK
1996Ireland
1995Norway
1994Ireland
1993Ireland
1992Ireland
1991Sweden
1990Italy
1989Yugoslavia
1988Switzerland
1987Ireland
1986Belgium
1985Norway
1984Sweden
1983Luxembourg
1982Germany
1981United KingdomUK
1980Ireland
1979Israel
1978Israel
1977France
1976United KingdomUK
1975Netherlands
1974Sweden
1973Luxembourg
1972Luxembourg
1971Monaco
1970Ireland
1969FranceNetherlandsSpainUnited KingdomUK
1968Spain
1967United KingdomUK
1966Austria
1965Luxembourg
1964Italy
1963Denmark
1962France
1961Luxembourg
1960France
1959Netherlands
1958France
1957Netherlands
1956Switzerland
['none', "Welcome to the show, let everybody know\nI'm done playing the game, I'll brea

In [11]:
euro.iloc[1]['Cleaned_Year']

'2024'

In [60]:
lyrics_series = pd.Series(lyrics_list)

In [61]:
euro['Lyrics'] = lyrics_series

In [63]:
euro.to_csv('euro_with_lyrics.csv')

In [64]:
test = pd.read_csv('euro_with_lyrics.csv')

In [65]:
test

,Unnamed: 0,Country,Song,Artist,Points,Cleaned_Year,Lyrics
0,0,—,—,NaN,—,2025,none
1,1,Switzerland,The Code,Nemo,591,2024,"Welcome to the show, let everybody know\nI'm d..."
2,2,Sweden,Tattoo,Loreen,583,2023,I don't wanna go\nBut baby we both know\nThis ...
3,3,Ukraine,Stefania,Kalush Orchestra,631,2022,Stefania mother mother Stefania\nThe field blo...
4,4,Italy,Zitti e buoni,Måneskin,524,2021,They don't know what I am talkin' about\nDirty...
...,...,...,...,...,...,...,...
65,65,France,Tom Pillibi,Jacqueline Boyer,32,1960,Tom Pillibi has two castles – the first one in...
66,66,Netherlands,Een beetje,Teddy Scholten,21,1959,I wanted your heart to be a closet with a litt...
67,67,France,Dors mon amour,André Claveau,27,1958,"Sleep, my love\nThe sun is still far from the ..."
68,68,Netherlands,Net als toen,Corry Brokken,31,1957,Don't be so drowsy with that eternal paper\nDo...
